In [1]:
import sys
import time
import csv
import math
import numpy as np
import pyspark
from pyspark import SparkContext, SparkConf
import json
from operator import add
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from node2vec import Node2Vec as n2v
import networkx as nx
from hashlib import md5
import pickle
import os
import pandas as pd
from xgboost import XGBRegressor

pd.set_option("display.max_columns", 100)
pd.set_option('display.width', 100)
pd.set_option("display.precision", 2)


from utils import create_category_md5_mapping, integrate_mapping_user_bus_cat_data, dataframe_to_rdd_dict, analyze_top_business_categories, analyze_top_categories

import json
import numpy as np
from pyspark import SparkContext, SparkConf
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

def initialize_spark_context(APP_NAME="Train: XGBModel"):
    # Spark配置项列表
    SPARK_CONF = [
        ("spark.dynamicAllocation.enabled", "true"),  # 启用动态资源分配
        ("spark.dynamicAllocation.maxExecutors", "2"),  # 最大执行器数量
        ("spark.executor.memory", "4g"),  # 每个执行器的内存
        ("spark.executor.cores", "6"),  # 每个执行器的CPU核心数
        ("spark.executor.memoryOverhead", "3000"),  # 执行器内存开销
        ("spark.driver.memory", "4g"),  # 驱动程序的内存
        ("spark.driver.maxResultSize", "3"),  # 驱动程序的最大结果大小
        ("spark.python.worker.memory", "4g"),  # Python工作进程的内存
        ("spark.sql.shuffle.partitions", "20"),  # Shuffle操作的分区数
        ("spark.sql.sources.partitionOverWriteMode", "dynamic"),  # 分区覆写模式
        # ("spark.network.timeout", "10s"),  # 网络超时设置
        # ("spark.executor.heartbeatInterval", "10s"),  # 执行器心跳间隔
    ]

    # 创建Spark配置
    spark_conf = pyspark.SparkConf()
    spark_conf.setAppName(APP_NAME)
    spark_conf.setAll(SPARK_CONF)

    # 创建SparkContext
    sc = pyspark.SparkContext(conf=spark_conf)
    sc.setLogLevel("ERROR")  # 设置日志级别

    return sc

sc = initialize_spark_context()

24/05/03 14:09:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/03 14:09:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import better_features
from better_features import FeatureProcessor, read_json_data, transform_user_data, transform_business_data, extract_review_data
from datetime import datetime #add
import time
import pandas as pd

def rdd_to_pandas(rdd):
    return pd.DataFrame(rdd.collect(), columns=rdd.first().keys())

def prepare_test_data(test_df, cluster):
    if cluster == -1:
        test_cluster_data = test_df
    else:
        test_cluster_data = test_df[test_df['Cluster'] == cluster]
        
    X_test = test_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
    y_test = test_cluster_data['stars']
    return X_test, y_test

start_time = time.time()

user_parsed_df = pd.read_csv('cache/user_df.csv') # parsed from users.json
business_parsed_df = pd.read_csv('cache/business_df.csv') # parsed from business.json
review_parsed_df = pd.read_csv('cache/review_df.csv') # parsed from business.json

train_df = pd.read_csv('cache/train_df.csv', index_col=None)
test_df = pd.read_csv('cache/test_df.csv', index_col=None)
val_df = pd.read_csv('cache/val_df.csv', index_col=None)

print("DONE!")

DONE!


In [ ]:
XGBOOST

3
Best parameters: {'learning_rate': 0.08288064461501718, 'max_depth': 4.704925810006415, 
                  'n_estimators': 304.68373839140463, 'subsample': 0.6838579015297168}

-0.9463

4
Best parameters: {'learning_rate': 0.18299396047960448, 'max_depth': 3.717897930418724, 
                  'n_estimators': 249.53928852131315, 'subsample': 0.9081495526065285}
-0.9327

2
Best parameters: {'learning_rate': 0.12350751460907078, 'max_depth': 3.097672114006384, 
                  'n_estimators': 288.4233026512157, 'subsample': 0.8880304987479203}
-0.8718


0
Best parameters: {'learning_rate': 0.04246778091879101, 'max_depth': 6.332694494015188, 
                  'n_estimators': 491.29196986785007, 'subsample': 0.8689265095175513}
-0.9824

In [ ]:
CatBoost

3
Best parameters found: {'depth': 4.618128240457896, 'l2_leaf_reg': 0.23581627180328404, 
                        'learning_rate': 0.19681345553197774, 'n_estimators': 61.862927823711985}
-0.9447

4
Best parameters found: {'depth': 4.1198653673336825, 'l2_leaf_reg': 0.33761517140362796, 
                        'learning_rate': 0.19143645558687788, 'n_estimators': 45.856234561660415}
-0.9318

2
Best parameters found: {'depth': 6.1686286238462005, 'l2_leaf_reg': 28.02316020981978, 
                        'learning_rate': 0.10908927606829158, 'n_estimators': 176.43343531650365}
-0.8713

# Best parameters found: {'depth': 6.467060867155791, 'l2_leaf_reg': 17.065903814470666, 
#                         'learning_rate': 0.3860353293082986, 'n_estimators': 145.27827823278525}
# # -0.8712

0
Best parameters found: {'depth': 12.599856656197252, 'l2_leaf_reg': 18.150895075720104, 
                        'learning_rate': 0.06229424029098117, 'n_estimators': 1486.2454903402163}
-0.9821

In [ ]:
Best Cat/Xgb ratio for cluster 0: 0.30 with RMSE: 0.9815215776560651
Best Cat/Xgb ratio for cluster 2: 0.65 with RMSE: 0.8709046585277763

no improvement for 3/4, use catboost only w/ train-val split

In [ ]:
['score', 'average_stars', 'user_review_count', 'fans', 
 'useful', 'funny', 'cool', 'num_interactions', 'yelping_since', 
 'friends_count', 'elite_years', 'compliments', 'compliment_hot', 
 'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 
 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_writer', 
 'compliment_photos', 'total_ufc_count_per_year', 'avg_ufc_count_per_review', 
 'bus_stars', 'bus_review_count', 'is_open', 
 'city_encoded', 'log_affinity_score', 'binary_affinity_score']

In [ ]:
# 5,7,8, 6

In [ ]:
Best RMSEs for each cluster: {5: 1.016683932307932, 
                              7: 1.0235736207689943,
                              8: 1.0052618567763993}

Best RMSEs for each cluster: {6: 0.8837446917922023}


Best Parameters for each cluster: {5: {'depth': 2.8372035559550515, 'l2_leaf_reg': 0.415375550656062, 'learning_rate': 0.9017553809036529}, 
                                   7: {'depth': 5.595277607917133, 'l2_leaf_reg': 18.3792029910461, 'learning_rate': 0.9372003134293689}, 
                                   8: {'depth': 1.259435014043129, 'l2_leaf_reg': 7.440376345058953, 'learning_rate': 0.4049807340854126}}

Best Parameters for each cluster: {6: {'depth': 2.0, 'l2_leaf_reg': 1.1835517768726047, 'learning_rate': 0.38770112704134657}}



In [ ]:
test_size = 0.05

Cluster 5 RMSE: 1.0538654069790576
Cluster 5 Important features: 
['cool', 'compliment_writer', 'bus_stars']
Cluster 7 RMSE: 0.7757271895339214
Cluster 7 Important features: 
['score', 'average_stars', 'user_review_count', 'fans', 'useful', 'funny', 'cool', 'num_interactions', 'yelping_since', 'friends_count', 'elite_years', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_writer', 'compliment_photos', 'total_ufc_count_per_year', 'avg_ufc_count_per_review', 'bus_stars', 'bus_review_count', 'is_open', 'city_encoded', 'log_affinity_score']
Cluster 8 RMSE: 1.0078177838379447
Cluster 8 Important features: 
['average_stars', 'num_interactions', 'total_ufc_count_per_year', 'bus_stars']

test_size = 0.1

Cluster 5 RMSE: 1.0282390060477125
Cluster 5 Important features: ['cool', 'elite_years', 'compliment_writer',
                               'avg_ufc_count_per_review', 'bus_stars', 'city_encoded']
Cluster 7 RMSE: 0.6417139710941048
Cluster 7 Important features: ['score', 'average_stars', 'user_review_count', 
                               'fans', 'useful', 'funny', 'cool', 'num_interactions', 
                               'yelping_since', 'friends_count', 'elite_years', 'compliments', 
                               'compliment_hot', 'compliment_more', 'compliment_profile', 
                               'compliment_note', 'compliment_plain', 'compliment_cool', 
                               'compliment_writer', 'compliment_photos', 'total_ufc_count_per_year',
                               'avg_ufc_count_per_review', 'bus_stars', 'bus_review_count', 'is_open', 
                               'city_encoded', 'log_affinity_score']
Cluster 8 RMSE: 1.009791395972113
Cluster 8 Important features: ['average_stars', 'compliments', 'bus_stars']

Cluster 6 RMSE: 0.8325986068472191
Cluster 6 Important features: ['score', 'average_stars', 'user_review_count', 'fans', 'useful', 'funny', 'cool', 'num_interactions', 'elite_years', 'compliments', 'compliment_profile', 'compliment_cute', 'compliment_writer', 'bus_stars', 'bus_review_count', 'is_open', 'city_encoded', 'log_affinity_score']

In [ ]:
'''removing features'''
'log_affinity_score'
Cluster 5 RMSE: 0.9055991155707416

'score', 'log_affinity_score'
Cluster 8 RMSE: 0.9846981283045735

'score', 'log_affinity_score'
Cluster 7 RMSE: 0.6407088305737814

'/'
Cluster 6 RMSE: 0.8325986068472191

In [ ]:

# 'log_affinity_score', 'binary_affinity_score'
# Cluster 5 RMSE: 0.9825581813966131

# 'score'
# Cluster 8 RMSE: 0.9941245480634432

In [ ]:
ridge
test_size=0.5
model = RidgeCV(alphas=np.logspace(-6, 6, 13), cv=KFold(5))
Cluster 1 RMSE: 0.9339884924736819


elasticNet
test_size=0.05
Best alpha: 0.010974987654930568
Best l1_ratio: 0.1
Cluster 1 RMSE: 0.9662513300113994


In [ ]:
# 总结的模型性能和参数设置
ALL use early_stopping_rounds=10

xgb_params = {
    0: {'learning_rate': 0.04246778091879101, 'max_depth': 6, 'n_estimators': 491, 'subsample': 0.8689},
    2: {'learning_rate': 0.12350751460907078, 'max_depth': 3, 'n_estimators': 288, 'subsample': 0.8880},
    3: {'learning_rate': 0.08288064461501718, 'max_depth': 5, 'n_estimators': 305, 'subsample': 0.6839},
    4: {'learning_rate': 0.18299396047960448, 'max_depth': 4, 'n_estimators': 250, 'subsample': 0.9081}
}

catboost_params = {
    0: {'depth': 12, 'l2_leaf_reg': 18.15, 'learning_rate': 0.06229, 'n_estimators': 1486},
    2: {'depth': 6, 'l2_leaf_reg': 28.02, 'learning_rate': 0.1090, 'n_estimators': 176},
    3: {'depth': 5, 'l2_leaf_reg': 0.2358, 'learning_rate': 0.1968, 'n_estimators': 62},
    4: {'depth': 4, 'l2_leaf_reg': 0.3376, 'learning_rate': 0.1914, 'n_estimators': 46}
}

Best Cat/Xgb ratio for cluster 0: 0.30 with RMSE: 0.9815215776560651
Best Cat/Xgb ratio for cluster 2: 0.65 with RMSE: 0.8709046585277763

no improvement for 3/4, use catboost only w/ train-val split

Cluster 0:
模型: CatBoost + XGBoost (no val)
RMSE: 0.9815215776560651

Cluster 2:
模型: CatBoost + XGBoost (no val)
RMSE: 0.8709046585277763

3,4 uses test_size = 0.05
Cluster 3:
模型: CatBoost
RMSE: 0.9447
Cluster 4:
模型: CatBoost
RMSE: 0.9318
====================
best_params = {
    5: {'depth': 3, 'l2_leaf_reg': 0.415375550656062, 'learning_rate': 0.9017553809036529}, 
    7: {'depth': 6, 'l2_leaf_reg': 18.3792029910461, 'learning_rate': 0.9372003134293689}, 
    8: {'depth': 1, 'l2_leaf_reg': 7.440376345058953, 'learning_rate': 0.4049807340854126},
    6: {'depth': 2.0, 'l2_leaf_reg': 1.1835517768726047, 'learning_rate': 0.38770112704134657}
}

'''removing features'''
'log_affinity_score'
Cluster 5 RMSE: 0.9055991155707416

'score', 'log_affinity_score'
Cluster 8 RMSE: 0.9846981283045735

Cluster 7 RMSE: 0.6407088305737814

'/'
Cluster 6 RMSE: 0.8325986068472191

Cluster 5, 6, 7, 8:
模型: CatBoost
参数: 各自最优
RMSE
:0.9055991155707416
:0.8325986068472191
:0.6407088305737814
:0.9846981283045735
======================
Cluster 1:
模型: ElasticNet
参数: alpha 0.010974987654930568, l1_ratio 0.1
RMSE: 0.9662513300113994